In [2]:
import os
import sys

import pandas as pd
import numpy as np

import torch
import tensorflow as tf
import keras

from torch.utils.data import Subset
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler

from utils.TSDataset import TimeSeriesDataset
from utils.TSDataset import data_load
from utils.plot import *
from utils.split_train_val_test import *
from utils.compute_metric import compute_metrics, append_score
from utils.compute_metric import compute_metrics_seq2seq
from utils.models import cnn_lstm_model_eval

import matplotlib as mpl
import matplotlib.pyplot as plt


In [3]:
project_dir = os.path.dirname(os.path.abspath('__file__'))
project_dir

data_path = os.path.join(project_dir, 'data', 'processed', 'BTC-USD-sample.csv')

# Load data
data1, x_scaler1, y_scaler1 = data_load(data_path, x_scaler='minmax', y_scaler='minmax')

# Multi-Step Prediction

In [4]:
record = []

In [14]:
forecast_horizons = [1, 10, 60]
train_df, test_df = split_train_val_test(data1, train_frac=0.7)
scores = []
target_col = 'y'

test_target = test_df[target_col].values

for h in forecast_horizons:
    naive_y_true = []  # will hold the true future values for every forecasting window
    naive_y_pred = []  # will hold the corresponding naive predictions
    
    for i in range(len(test_target) - h):
        # True values: for instance, at time step 0, this gets indices 1 to h (i.e., 1:11 when h=10)
        y_true = test_target[i + 1: i + h + 1]
        # Naive predictions: create an array of length h filled with the value at time step i
        y_pred = np.full((h,), test_target[i])
        
        naive_y_true.append(y_true)
        naive_y_pred.append(y_pred)
    
    naive_y_true = y_scaler1.inverse_transform(naive_y_true)
    naive_y_pred = y_scaler1.inverse_transform(naive_y_pred)
    mse_naive, mae_naive, huber_naive = compute_metrics(naive_y_true, naive_y_pred)
    
    # Print out the performance for this forecast horizon
    record = {
        'model': 'Naive',
        'h-step Forecast': h,
        'mse': mse_naive,
        'mae': mae_naive,
        'huber': huber_naive
    }
    append_score(scores, record)
for i in scores:
    print(i)

{'model': 'Naive', 'h-step Forecast': 1, 'mse': 1160.3737790527032, 'mae': 25.10380253502343, 'huber': np.float32(24.65289)}
{'model': 'Naive', 'h-step Forecast': 10, 'mse': 5275.25768559061, 'mae': 54.060173825503384, 'huber': np.float32(53.572163)}
{'model': 'Naive', 'h-step Forecast': 60, 'mse': 26082.668655180554, 'mae': 116.48661504629631, 'huber': np.float32(115.99019)}


In [6]:
window_size = 100
target_col = 'y'
feature_cols = ['x1', 'x2', 'x3', 'x4', 'x5']
return_index = True

In [11]:
data_frame = data1
feature_cols = ['x1', 'x2', 'x3', 'x4', 'x5']
target_col = 'y'
return_index = True
scaler = y_scaler1
filters = 26
window_size = 129
kernel_size = 2
strides = 1
lstm_units = 31
learning_rate = 0.01835
epochs = 87
forecast_horizon = [1, 10, 60]
seq2seq = True

for h in forecast_horizon:
    mse = -cnn_lstm_model_eval(data_frame = data_frame, 
                        feature_cols = feature_cols, 
                        target_col = target_col, 
                        return_index = return_index, 
                        scaler = scaler, 
                        filters = filters, 
                        window_size = window_size, 
                        kernel_size = kernel_size, 
                        strides = strides, 
                        lstm_units = lstm_units, 
                        learning_rate = learning_rate, 
                        epochs = epochs, 
                        seq2seq = seq2seq, 
                        forecast_horizon = h)
    print(f"MSE for {h}-step forecast: {mse}")



43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
MSE for 1-step forecast: 339.7964571708816
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
MSE for 10-step forecast: 8107.401599442298
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
MSE for 60-step forecast: 50929.79013949266


In [6]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

for h in forecast_horizons:
    train_df, test_df = split_train_val_test(data1, train_frac=0.7)
    train_set = TimeSeriesDataset(dataframe=train_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)
    test_set = TimeSeriesDataset(dataframe=test_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)

    X_train, y_train, x_dates, y_dates = train_set.X_seq, train_set.y_seq, train_set.x_dates, train_set.y_dates
    X_test, y_test, x_dates_test, y_dates_test = test_set.X_seq, test_set.y_seq, test_set.x_dates, test_set.y_dates

    dense_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(window_size, len(feature_cols))),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(h)
    ])

    dense_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    history_dense = dense_model.fit(X_train, y_train, epochs=50,
                                    validation_data=(X_test, y_test), verbose=0)

    #plot_learning_curves(history_dense.history)
    y_pred = dense_model.predict(X_test)
    y_pred = y_scaler1.inverse_transform(y_pred.reshape(-1, 1))
    y_true = y_scaler1.inverse_transform(y_test.reshape(-1, 1))
    
    mse_dense, mae_dense, huber_dense = compute_metrics(y_true, y_pred)
    
    record = {
        'model': 'Dense',
        'h-step Forecast': h,
        'mse': mse_dense,
        'mae': mae_dense,
        'huber': huber_dense
    }
    append_score(scores, record)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step


In [7]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

for h in forecast_horizons:
    train_df, test_df = split_train_val_test(data1, train_frac=0.7)
    train_set = TimeSeriesDataset(dataframe=train_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)
    test_set = TimeSeriesDataset(dataframe=test_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)

    X_train, y_train, x_dates, y_dates = train_set.X_seq, train_set.y_seq, train_set.x_dates, train_set.y_dates
    X_test, y_test, x_dates_test, y_dates_test = test_set.X_seq, test_set.y_seq, test_set.x_dates, test_set.y_dates

    rnn_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(window_size, len(feature_cols))),
        tf.keras.layers.SimpleRNN(units=20),
        tf.keras.layers.Dense(h)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    rnn_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    history_rnn = rnn_model.fit(X_train, y_train, epochs=20,
                                validation_data=(X_test, y_test), verbose=0)

    y_pred = rnn_model.predict(X_test)
    y_pred = y_scaler1.inverse_transform(y_pred.reshape(-1, 1))
    y_true = y_scaler1.inverse_transform(y_test.reshape(-1, 1))
    
    mse_rnn, mae_rnn, huber_rnn = compute_metrics(y_true, y_pred)
    
    record = {
        'model': 'Simple RNN',
        'h-step Forecast': h,
        'mse': mse_rnn,
        'mae': mae_rnn,
        'huber': huber_rnn
    }
    scores.append(record)
    #plot_learning_curves(history_rnn.history)

/Users/jeddyxie/CNN-LSTM/CNN-LSTM/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


/Users/jeddyxie/CNN-LSTM/CNN-LSTM/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


/Users/jeddyxie/CNN-LSTM/CNN-LSTM/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


/Users/jeddyxie/CNN-LSTM/CNN-LSTM/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [8]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

for h in forecast_horizons:
    train_df, test_df = split_train_val_test(data1, train_frac=0.7)
    train_set = TimeSeriesDataset(dataframe=train_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)
    test_set = TimeSeriesDataset(dataframe=test_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)

    X_train, y_train, x_dates, y_dates = train_set.X_seq, train_set.y_seq, train_set.x_dates, train_set.y_dates
    X_test, y_test, x_dates_test, y_dates_test = test_set.X_seq, test_set.y_seq, test_set.x_dates, test_set.y_dates

    rnn_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(window_size, len(feature_cols))),
    tf.keras.layers.SimpleRNN(units=20, return_sequences=True),
    tf.keras.layers.SimpleRNN(units=20, return_sequences=False),
    tf.keras.layers.Dense(h)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    rnn_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    history_rnn = rnn_model.fit(X_train, y_train, epochs=20,
                                validation_data=(X_test, y_test), verbose=0)

    y_pred = rnn_model.predict(X_test)
    y_pred = y_scaler1.inverse_transform(y_pred.reshape(-1, 1))
    y_true = y_scaler1.inverse_transform(y_test.reshape(-1, 1))
    
    mse_rnn, mae_rnn, huber_rnn = compute_metrics(y_true, y_pred)
    
    record = {
        'model': 'Deep RNN',
        'h-step Forecast': h,
        'mse': mse_rnn,
        'mae': mae_rnn,
        'huber': huber_rnn
    }
    append_score(scores, record)
    #plot_learning_curves(history_rnn.history)

/Users/jeddyxie/CNN-LSTM/CNN-LSTM/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


/Users/jeddyxie/CNN-LSTM/CNN-LSTM/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


/Users/jeddyxie/CNN-LSTM/CNN-LSTM/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


/Users/jeddyxie/CNN-LSTM/CNN-LSTM/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [9]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

for h in forecast_horizons:
    train_df, test_df = split_train_val_test(data1, train_frac=0.7)
    train_set = TimeSeriesDataset(dataframe=train_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)
    test_set = TimeSeriesDataset(dataframe=test_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)

    X_train, y_train, x_dates, y_dates = train_set.X_seq, train_set.y_seq, train_set.x_dates, train_set.y_dates
    X_test, y_test, x_dates_test, y_dates_test = test_set.X_seq, test_set.y_seq, test_set.x_dates, test_set.y_dates

    rnn_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(window_size, len(feature_cols))),
    tf.keras.layers.SimpleRNN(units=20, return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.SimpleRNN(units=20, return_sequences=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(h)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    rnn_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    history_rnn = rnn_model.fit(X_train, y_train, epochs=20,
                                validation_data=(X_test, y_test), verbose=0)

    y_pred = rnn_model.predict(X_test)
    y_pred = y_scaler1.inverse_transform(y_pred.reshape(-1, 1))
    y_true = y_scaler1.inverse_transform(y_test.reshape(-1, 1))
    
    mse_rnn, mae_rnn, huber_rnn = compute_metrics(y_true, y_pred)
    
    record = {
        'model': 'RNN Batch Normalization',
        'h-step Forecast': h,
        'mse': mse_rnn,
        'mae': mae_rnn,
        'huber': huber_rnn
    }
    append_score(scores, record)
    #plot_learning_curves(history_rnn.history)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


In [10]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

for h in forecast_horizons:
    train_df, test_df = split_train_val_test(data1, train_frac=0.7)
    train_set = TimeSeriesDataset(dataframe=train_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)
    test_set = TimeSeriesDataset(dataframe=test_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)

    X_train, y_train, x_dates, y_dates = train_set.X_seq, train_set.y_seq, train_set.x_dates, train_set.y_dates
    X_test, y_test, x_dates_test, y_dates_test = test_set.X_seq, test_set.y_seq, test_set.x_dates, test_set.y_dates

    rnn_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(window_size, len(feature_cols))),
    tf.keras.layers.LSTM(units=20, return_sequences=True),
    tf.keras.layers.LSTM(units=20, return_sequences=False),
    tf.keras.layers.Dense(h)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    rnn_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    history_rnn = rnn_model.fit(X_train, y_train, epochs=20,
                                validation_data=(X_test, y_test), verbose=0)

    y_pred = rnn_model.predict(X_test)
    y_pred = y_scaler1.inverse_transform(y_pred.reshape(-1, 1))
    y_true = y_scaler1.inverse_transform(y_test.reshape(-1, 1))
    
    mse_rnn, mae_rnn, huber_rnn = compute_metrics(y_true, y_pred)
    
    record = {
        'model': 'LSTM',
        'h-step Forecast': h,
        'mse': mse_rnn,
        'mae': mae_rnn,
        'huber': huber_rnn
    }
    append_score(scores, record)
    #plot_learning_curves(history_rnn.history)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [11]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
forecast_horizons = [10]
for h in forecast_horizons:
    train_df, test_df = split_train_val_test(data1, train_frac=0.7)
    train_set = TimeSeriesDataset(dataframe=train_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)
    test_set = TimeSeriesDataset(dataframe=test_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)

    X_train, y_train, x_dates, y_dates = train_set.X_seq, train_set.y_seq, train_set.x_dates, train_set.y_dates
    X_test, y_test, x_dates_test, y_dates_test = test_set.X_seq, test_set.y_seq, test_set.x_dates, test_set.y_dates

    rnn_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(window_size, len(feature_cols))),
    tf.keras.layers.GRU(units=20, return_sequences=True),
    tf.keras.layers.GRU(units=20, return_sequences=False),
    tf.keras.layers.Dense(h)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    rnn_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    history_rnn = rnn_model.fit(X_train, y_train, epochs=3,
                                validation_data=(X_test, y_test), verbose=0)

    y_pred = rnn_model.predict(X_test)

    y_pred = y_scaler1.inverse_transform(y_pred.reshape(-1, 1))
    y_true = y_scaler1.inverse_transform(y_test.reshape(-1, 1))
    
    mse_rnn, mae_rnn, huber_rnn = compute_metrics(y_true, y_pred)
    print(mse_rnn)
    print(mae_rnn)
    print(huber_rnn)
    
    record = {
        'model': 'GRU',
        'h-step Forecast': h,
        'mse': mse_rnn,
        'mae': mae_rnn,
        'huber': huber_rnn
    }
    append_score(scores, record)
    #plot_learning_curves(history_rnn.history)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
8817.181446333696
73.64343211321192
73.14473


In [15]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 120
for h in forecast_horizons:
    train_df, test_df = split_train_val_test(data1, train_frac=0.7)
    train_set = TimeSeriesDataset(dataframe=train_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)
    test_set = TimeSeriesDataset(dataframe=test_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)

    X_train, y_train, x_dates, y_dates = train_set.X_seq, train_set.y_seq, train_set.x_dates, train_set.y_dates
    X_test, y_test, x_dates_test, y_dates_test = test_set.X_seq, test_set.y_seq, test_set.x_dates, test_set.y_dates

   
    cnn_model = keras.models.Sequential([
    keras.layers.Input(shape=(window_size, len(feature_cols))),
    keras.layers.Conv1D(filters=16, kernel_size=2, padding="valid"),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32, return_sequences=False),
    keras.layers.Dense(h)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.004717880792838919)
    cnn_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    history_rnn = cnn_model.fit(X_train, y_train, epochs=20,
                                validation_data=(X_test, y_test), verbose=0)

    y_pred = cnn_model.predict(X_test)
    y_pred = y_scaler1.inverse_transform(y_pred.reshape(-1, 1))
    y_true = y_scaler1.inverse_transform(y_test.reshape(-1, 1))
    
    mse_rnn, mae_rnn, huber_rnn = compute_metrics(y_true, y_pred)
    
    cnn_lstm_record = {
        'model': 'CNN-LSTM',
        'h-step Forecast': h,
        'mse': mse_rnn,
        'mae': mae_rnn,
        'huber': huber_rnn
    }
    append_score(scores, cnn_lstm_record)
    #plot_learning_curves(history_rnn.history)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


In [16]:
for i in scores:
    print(i)

{'model': 'Naive', 'h-step Forecast': 120, 'mse': 47600.81892224578, 'mae': 160.01141008454107, 'huber': np.float32(159.51382)}
{'model': 'Naive', 'h-step Forecast': 240, 'mse': 92012.92070534195, 'mae': 222.9653111772487, 'huber': np.float32(222.46693)}
{'model': 'Naive', 'h-step Forecast': 360, 'mse': 146950.88944180385, 'mae': 293.7249538255361, 'huber': np.float32(293.2261)}
{'model': 'CNN-LSTM', 'h-step Forecast': 120, 'mse': 47717.291397079745, 'mae': 156.00167939646272, 'huber': np.float32(155.50249)}
{'model': 'CNN-LSTM', 'h-step Forecast': 240, 'mse': 106428.28298186195, 'mae': 234.52212818255597, 'huber': np.float32(234.02264)}
{'model': 'CNN-LSTM', 'h-step Forecast': 360, 'mse': 254819.54339690704, 'mae': 388.6029576857995, 'huber': np.float32(388.10324)}


In [24]:
from bayes_opt import BayesianOptimization

window_size = 100
forecast_horizons = 1

train_df, test_df = split_train_val_test(data1, train_frac=0.7)
train_set = TimeSeriesDataset(dataframe=train_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)
test_set = TimeSeriesDataset(dataframe=test_df, window_size=window_size, forecast_horizon=h, feature_cols=feature_cols, target_col=target_col, return_index=return_index)

X_train, y_train, x_dates, y_dates = train_set.X_seq, train_set.y_seq, train_set.x_dates, train_set.y_dates
X_test, y_test, x_dates_test, y_dates_test = test_set.X_seq, test_set.y_seq, test_set.x_dates, test_set.y_dates


cnn_model = keras.models.Sequential([
keras.layers.Input(shape=(window_size, len(feature_cols))),
keras.layers.Conv1D(filters=16, kernel_size=2, padding="valid"),
keras.layers.LSTM(32, return_sequences=True),
keras.layers.LSTM(32, return_sequences=False),
keras.layers.Dense(h)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.004717880792838919)
cnn_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

history_rnn = cnn_model.fit(X_train, y_train, epochs=20,
                validation_data=(X_test, y_test), verbose=0)

In [ ]:
y_pred = cnn_model.predict(X_test)
y_pred = y_scaler1.inverse_transform(y_pred)
y_true = y_scaler1.inverse_transform(y_test)

mse_rnn, mae_rnn, huber_rnn = compute_metrics(y_true, y_pred)

cnn_lstm_record = {
'model': 'CNN-LSTM',
'h-step Forecast': h,
'mse': mse_rnn,
'mae': mae_rnn,
'huber': huber_rnn
}
append_score(scores, cnn_lstm_record)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
